In [1]:
# =============================================================================

# Folder: International_Trade_Analysis_Assignment/notebooks/Trade_Data_Pipeline.ipynb
# =============================================================================

import pandas as pd
import numpy as np
import re
import os
import sqlite3
from IPython.display import display, FileLink

print("Trade Data Pipeline Started - 100% Working Version")

Trade Data Pipeline Started - 100% Working Version


In [4]:
# CELL 1: Load Raw Data – WORKS 100% with your current folder structure
import pandas as pd

# Use this path when notebook is inside "notebooks" folder
df = pd.read_excel("../data/raw/Sample Data 2.xlsx")

print(f"Raw data loaded successfully!")
print(f"Shape: {df.shape[0]} rows × {df.shape[1]} columns")
df.head(10)

Raw data loaded successfully!
Shape: 13 rows × 10 columns


,Date of Shipment,HSN Code,Goods Description,Quantity,Unit,Unit Price (INR),Total Value (INR),Duty Paid (INR),Supplier Name and Address,HSN Description
0,15/03/2017,70134900,BOROSILICATE GLASS TUMBLER 350ML MODEL NO BT35...,1200,PCS,85.20,102240,28507,"SHANGHAI GLASSWARE CO LTD, CHINA",Glassware of a kind used for table/kitchen
1,22/06/2018,44199090,WOODEN SPOON 12 INCH PACK OF 100 @ USD 0.35/PC,500,NOS,29.75,14875,4162,"YIWU WOODCRAFT FACTORY, CHINA",Articles of wood
2,10/09/2019,73239390,STAINLESS STEEL BOWL 500ML SS-501 @ USD 2.8/PC,800,PCS,238.00,190400,53292,"GUANGZHOU STEEL PRODUCTS LTD, CHINA",Table/kitchenware of stainless steel
3,05/01/2020,39241090,PLASTIC WATER BOTTLE 1000ML PB-1000 USD 0.95 P...,2000,PCS,80.75,161500,45162,"NINGBO PLASTIC MOULD CO, CHINA",Table/kitchenware of plastics
4,18/04/2021,84148090,GREENHOUSE FAN 50 INCH HEAVY DUTY USD 185/PC,50,NOS,15725.00,786250,219918,"QINGDAO GREENHOUSE TECH CO LTD, CHINA",Air or vacuum pumps
5,30/07/2022,70134900,BOROSILICATE GLASS JAR 1000ML BJ-1000 @ USD 2....,600,PCS,178.50,107100,29944,"SHANDONG BOROSILICATE GLASS, CHINA",Glassware of a kind used for table/kitchen
6,12/11/2023,70134900,OPALWARE DINNER SET 36 PCS OP-36 @ USD 18/SET,300,SET,1530.00,459000,128367,"GUANGDONG OPALWARE MANUFACTURER, CHINA",Glassware of a kind used for table/kitchen
7,25/02/2024,44199090,BAMBOO FORK & SPOON SET PACK OF 50 USD 0.45/PC,1000,NOS,38.25,38250,10699,"FUJIAN BAMBOO PRODUCTS CO, CHINA",Articles of wood
8,14/06/2024,73239390,SS CUTLERY SET 24 PCS MODEL CS-24 @ USD 8.5/SET,400,SET,722.50,289000,80812,"WUYI STAINLESS STEEL FACTORY, CHINA",Table/kitchenware of stainless steel
9,03/08/2025,70134900,BOROSILICATE GLASS BOTTLE 500ML BB500 @ USD 1....,1500,PCS,140.25,210375,58825,"HEBEI GLASS INDUSTRIES, CHINA",Glassware of a kind used for table/kitchen


In [5]:
# CELL 2: Basic Cleaning & Date Handling
df = df.copy()
df.columns = [col.strip().replace(" ", "_").replace("(INR)", "").strip() for col in df.columns]

df.rename(columns={
    'Date_of_Shipment': 'date_of_shipment',
    'HSN_Code': 'hsn_code',
    'Goods_Description': 'goods_description',
    'Quantity': 'quantity',
    'Unit': 'unit',
    'Unit_Price_': 'unit_price_inr',
    'Total_Value_': 'total_value_inr',
    'Duty_Paid_': 'duty_paid_inr',
    'Supplier_Name_and_Address': 'supplier_name',
    'HSN_Description': 'hsn_description'
}, inplace=True)

df['date_of_shipment'] = pd.to_datetime(df['date_of_shipment'], format='%d/%m/%Y', errors='coerce')
df['year'] = df['date_of_shipment'].dt.year
df['month'] = df['date_of_shipment'].dt.month
df['year_month'] = df['date_of_shipment'].dt.to_period('M')

print("Basic cleaning done")

Basic cleaning done


In [6]:
# CELL 3: Advanced Text Parsing (Goods Description)
def parse_goods_description(text):
    if not isinstance(text, str):
        return {k: None for k in ['model_name','model_number','capacity_spec','material_type','embedded_qty','unit_price_usd']}
    
    t = text.upper()
    res = {'model_name':None, 'model_number':None, 'capacity_spec':None, 'material_type':None, 'embedded_qty':None, 'unit_price_usd':None}

    # USD Price
    p = re.search(r'(?:USD|\$)\s*([\d,\.]+)', t)
    if p: res['unit_price_usd'] = float(p.group(1).replace(',',''))

    # Capacity
    c = re.search(r'(\d+(?:\.\d+)?)\s*(ML|L|INCH|CM|MM|KG|G)', t)
    if c: res['capacity_spec'] = c.group(0).strip()

    # Material
    mats = ['BOROSILICATE','OPAL','OPALWARE','STAINLESS STEEL','SS','WOOD','WOODEN','BAMBOO','PLASTIC','GLASS']
    for m in mats:
        if m in t:
            res['material_type'] = m.title().replace('Ss', 'Stainless Steel')
            break

    # Model Number
    mn = re.search(r'\b([A-Z]{2,6}[-/]?\d{2,6})\b', t)
    if mn: res['model_number'] = mn.group(1)

    # Model Name
    clean = re.sub(r'(USD|\$|@).*', '', t)
    clean = re.sub(r'\d+(ML|L|INCH).*', '', clean)
    words = [w for w in clean.split() if w.isalpha() and len(w)>2]
    if words: res['model_name'] = ' '.join(words[:3]).title()

    # Embedded Qty
    eq = re.search(r'(?:PACK OF|X|SET OF)\s*(\d+)', t)
    if eq: res['embedded_qty'] = int(eq.group(1))

    return res

parsed = df['goods_description'].apply(parse_goods_description)
parsed_df = pd.json_normalize(parsed)
df = pd.concat([df.reset_index(drop=True), parsed_df], axis=1)

print("Text parsing completed!")
df[['goods_description','model_name','model_number','capacity_spec','material_type','unit_price_usd']].head(8)

Text parsing completed!


,goods_description,model_name,model_number,capacity_spec,material_type,unit_price_usd
0,BOROSILICATE GLASS TUMBLER 350ML MODEL NO BT35...,Borosilicate Glass Tumbler,BT350,350ML,Borosilicate,1.20
1,WOODEN SPOON 12 INCH PACK OF 100 @ USD 0.35/PC,Wooden Spoon Inch,None,12 INCH,Wood,0.35
2,STAINLESS STEEL BOWL 500ML SS-501 @ USD 2.8/PC,Stainless Steel Bowl,SS-501,500ML,Stainless Steel,2.80
3,PLASTIC WATER BOTTLE 1000ML PB-1000 USD 0.95 P...,Plastic Water Bottle,PB-1000,1000ML,Plastic,0.95
4,GREENHOUSE FAN 50 INCH HEAVY DUTY USD 185/PC,Greenhouse Fan Inch,None,50 INCH,None,185.00
5,BOROSILICATE GLASS JAR 1000ML BJ-1000 @ USD 2....,Borosilicate Glass Jar,BJ-1000,1000ML,Borosilicate,2.10
6,OPALWARE DINNER SET 36 PCS OP-36 @ USD 18/SET,Opalware Dinner Set,OP-36,None,Opal,18.00
7,BAMBOO FORK & SPOON SET PACK OF 50 USD 0.45/PC,Bamboo Fork Spoon,None,None,Bamboo,0.45


In [7]:
# CELL 4: Unit Standardization + Feature Engineering
unit_map = {'PCS':'PCS','PC':'PCS','NOS':'PCS','PIECE':'PCS','SET':'SET'}
df['unit_standardized'] = df['unit'].str.upper().str.strip().map(unit_map).fillna('PCS')

df['grand_total_inr'] = df['total_value_inr'] + df['duty_paid_inr']
df['landed_cost_per_unit'] = np.where(df['quantity'] > 0, df['grand_total_inr'] / df['quantity'], np.nan)
df['duty_percentage'] = np.where(df['total_value_inr'] > 0, df['duty_paid_inr'] / df['total_value_inr'] * 100, np.nan)

print("Feature engineering done")

Feature engineering done


In [8]:
# CELL 5: Category & Sub-Category
def assign_category(row):
    d = f"{row['goods_description']} {row['hsn_description']}".upper()
    if any(x in d for x in ['GLASS','BOROSILICATE','OPAL','TUMBLER','JAR']): return 'Glassware'
    elif any(x in d for x in ['WOOD','WOODEN','BAMBOO']): return 'Wooden Products'
    elif any(x in d for x in ['STEEL','SS','STAINLESS']): return 'Steel Products'
    elif 'PLASTIC' in d: return 'Plastic Products'
    elif 'FAN' in d: return 'Machinery'
    else: return 'Others'

def assign_subcategory(row):
    d = row['goods_description'].upper()
    if row['category'] == 'Glassware':
        if 'BOROSILICATE' in d: return 'Borosilicate'
        if 'OPAL' in d: return 'Opalware'
    elif row['category'] == 'Wooden Products':
        if 'SPOON' in d: return 'Spoons'
        if 'FORK' in d: return 'Forks'
    return 'Others'

df['category'] = df.apply(assign_category, axis=1)
df['sub_category'] = df.apply(assign_subcategory, axis=1)

print("Category logic applied")

Category logic applied


In [9]:
# CELL 6: Save Cleaned Data (CORRECT PATH)
os.makedirs("../data/processed", exist_ok=True)

df.to_csv("../data/processed/trade_data_cleaned.csv", index=False, encoding='utf-8-sig')
df.to_excel("../data/processed/trade_data_cleaned.xlsx", index=False)

print("CLEANED FILES SAVED!")
display(FileLink("../data/processed/trade_data_cleaned.xlsx"))

CLEANED FILES SAVED!


C:\Users\Hi\Assignment\International_Trade_Analysis_Assignment\data\processed\trade_data_cleaned.xlsx

In [10]:
# CELL 7: Create SQLite Database (Power BI Ready)
df_sql = df.copy()
df_sql['year_month'] = df_sql['year_month'].astype(str)
df_sql['date_of_shipment'] = df_sql['date_of_shipment'].astype(str)

conn = sqlite3.connect("../trade_analysis.db")
df_sql.to_sql("shipments", conn, if_exists='replace', index=False)
conn.close()

print("trade_analysis.db created successfully → Ready for Power BI!")

trade_analysis.db created successfully → Ready for Power BI!


In [13]:

print("="*70)
print("YOU ARE 100% DONE WITH THE ASSIGNMENT!")
print("="*70)
print("Files created:")
print("   → ../data/raw/Sample Data 2.xlsx")
print("   → ../data/processed/trade_data_cleaned.xlsx")
print("   → ../trade_analysis.db")
print("   → This notebook")


print("="*70)

YOU ARE 100% DONE WITH THE ASSIGNMENT!
Files created:
   → ../data/raw/Sample Data 2.xlsx
   → ../data/processed/trade_data_cleaned.xlsx
   → ../trade_analysis.db
   → This notebook


In [15]:
import os
os.getcwd()


'C:\\Users\\Hi\\Assignment\\International_Trade_Analysis_Assignment\\notebooks'

In [16]:
import os

for root, dirs, files in os.walk('..', topdown=True):
    for name in files:
        if name.endswith('.db'):
            print(os.path.join(root, name))


..\trade_analysis.db


In [18]:
import shutil

shutil.copy(
    '../trade_analysis.db',
    'C:/Users/Hi/Desktop/trade_analysis.db'
)



'C:/Users/Hi/Desktop/trade_analysis.db'